In [ ]:
from datawand.parametrization import ConfigGenerator

# Initialize configuration generator

In [ ]:
cg = ConfigGenerator(pipeline_name='LNGraph')

# Setting default parameters for pipeline

In [ ]:
DEFAULTS = {}

#### Set your default parameters below:

In [ ]:
DEFAULTS["data_dir"] = "/mnt/idms/fberes/data/bitcoin_ln_research"
DEFAULTS["weight"] = None
DEFAULTS["weight_cols"] = [None, "num_channels", "capacity", "fee_base_msat", "fee_rate_milli_msat"]
DEFAULTS["snapshot_ids"] = list(range(9))

In [ ]:
DEFAULTS["experiment_id"] = "trial"
DEFAULTS["snapshot_id"] = 0
DEFAULTS["num_transactions"] = 6000
DEFAULTS["amount_sat"] = 50000
DEFAULTS["eps"] = 0.8 
DEFAULTS["day_interval"] = 7
DEFAULTS["drop_disabled"] = True
DEFAULTS["drop_low_cap"] = True
DEFAULTS["with_depletion"] = True

DEFAULTS["sim_dir"] = "2019-09-03_15:28:05_50000sat_k5000_aNone_dropFalse"

# Setting parameters for notebooks

In [ ]:
PARAMETERS = {}

In [ ]:
for nb_name in cg.pythonitem_names:
    PARAMETERS[nb_name] = []
print(PARAMETERS)

#### Set your parameters for each notebooks below: 

##### You can specify the ending of the names of the generated notebook clones, if you add an "\_\_ ending \_\_" parameter. (Otherwise they will end with \_CLONE\_1, \_CLONE\_2, etc.)
##### WARNING: if the name of the generated notebook clone is not an identifier, then luigi will NOT be able to run it.

## Centrality computation

for w in DEFAULTS["weight_cols"]:
    PARAMETERS["CentralityCalculator"].append({"weight":w})

## a.) Transaction simulation

In [ ]:
import datetime as dt

### i.) Transaction failure experiments

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 5
tx_num = 1000#50000#1000#2000#10000#20000#5000

for sat in [10000,25000,50000,100000,200000,500000]:
    drop_disabled = True
    for amount, num_transactions in [(sat,tx_num)]:
        for snap in range(7):
            for i in range(num_samples):
                PARAMETERS['RunSimulator'].append({"drop_low_cap":False, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i_keep_low_cap.%i" % (start_time, amount, num_transactions, i)})

### ii.) LNBIG tuning weekly

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 5

for sat in [10000,25000,50000,100000,200000,500000]:
    drop_disabled = True
    for amount, num_transactions in [(sat,6000)]:
        for snap in range(7):
            for i in range(num_samples):
                PARAMETERS['RunSimulator'].append({"day_interval":7, "snapshot_id":snap, "drop_disabled":drop_disabled, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i_drop%s.%i" % (start_time, amount, num_transactions, drop_disabled, i)})

### ii.) LNBIG tuning daily

start_time = dt.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
num_samples = 5

for amount, num_transactions in [(500000,6000)]:
    for snap in range(54):
        for i in range(num_samples):
            PARAMETERS['RunSimulator'].append({"day_interval":1, "snapshot_id":snap, "amount_sat":amount, "num_transactions":num_transactions, "experiment_id":"%s_%isat_k%i.%i" % (start_time, amount, num_transactions, i)})


## b.) Transaction results

sim_dirs = [
    "2019-09-05_14:35:13_10000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_25000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_50000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_100000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_200000sat_k6000_aNone_dropTrue",
    "2019-09-05_14:35:13_500000sat_k6000_aNone_dropTrue",
]

sim_dirs = [
    "2019-09-04_14:12:21_500000sat_k5000_aNone_dropFalse",
    "2019-09-04_14:12:21_200000sat_k5000_aNone_dropFalse",
    "2019-09-02_17:05:18_100000sat_k5000_aNone_dropFalse",
    "2019-09-03_15:28:05_50000sat_k5000_aNone_dropFalse",
    "2019-09-04_14:12:21_25000sat_k5000_aNone_dropFalse",
    "2019-09-04_14:12:21_10000sat_k5000_aNone_dropFalse"
]

sim_dirs = [
"2019-09-06_22:03:19_50000sat_k6000",
"2019-09-07_05:06:14_25000sat_k6000",
"2019-09-07_10:23:25_10000sat_k6000",
"2019-09-07_14:24:11_100000sat_k6000",
"2019-09-07_18:37:32_200000sat_k6000",
"2019-09-07_22:40:54_500000sat_k6000",
]

for d in sim_dirs:
    PARAMETERS['SimulationResults'].append({"sim_dir":d})

## c.) Node routing incomes

simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations_1days/"
snapshots = list(range(40))
experiments = [
    "2019-09-06_22:03:19_50000sat_k6000",
    "2019-09-07_05:06:14_25000sat_k6000",
    "2019-09-07_10:23:25_10000sat_k6000",
    "2019-09-07_14:24:11_100000sat_k6000",
    "2019-09-07_18:37:32_200000sat_k6000",
    "2019-09-07_22:40:54_500000sat_k6000"
]

for exp_id in experiments:
    PARAMETERS['NodeRoutingIncomeStats'].append({"sim_root_dir":simulation_dir, "snapshots":snapshots, "sim_res_dir":exp_id})

# Export parameters into pipeline configuration file

In [ ]:
cg.save_params(PARAMETERS, DEFAULTS)

cg.pipeline_cfg